In [ ]:
import random
import string

WORDS = ["python","hangman","challenge","developer","computer","programming","algorithm","function","variable","iteration"]

def choose_word():
    return random.choice(WORDS).upper()

def display_word(secret, correct):
    return " ".join([c if c in correct else "_" for c in secret])

def play():
    secret = choose_word()
    correct = set()
    wrong = set()
    max_wrong = 6
    while True:
        print()
        print(display_word(secret, correct))
        print(f"Wrong guesses ({len(wrong)}/{max_wrong}): {' '.join(sorted(wrong))}" if wrong else f"Wrong guesses (0/{max_wrong}):")
        guess = input("Enter a letter: ").strip().upper()
        if not guess or len(guess) != 1 or guess not in string.ascii_uppercase:
            print("Please enter a single alphabetic character.")
            continue
        if guess in correct or guess in wrong:
            print("You already guessed that letter.")
            continue
        if guess in secret:
            correct.add(guess)
            if all(c in correct for c in secret):
                print()
                print("Congratulations! You guessed the word:", secret)
                break
        else:
            wrong.add(guess)
            if len(wrong) >= max_wrong:
                print()
                print("Game over. The word was:", secret)
                break

if __name__ == "__main__":
    while True:
        play()
        again = input("Play again? (y/n): ").strip().lower()
        if not again or again[0] != "y":
            print("Thanks for playing.")
            break

In [ ]:
from collections import deque

def gen_successors(state, banana_loc, locations):
    monkey, box, height, has = state
    successors = []
    if not has:
        for loc in locations:
            if monkey != loc and height == 'ground':
                successors.append((f"move {monkey}->{loc}", (loc, box, height, has)))
        if monkey == box and height == 'ground':
            for dest in locations:
                if dest != monkey:
                    successors.append((f"push {monkey}->{dest}", (dest, dest, height, has)))
        if monkey == box and height == 'ground':
            successors.append(("climb", (monkey, box, 'on_box', has)))
        if height == 'on_box':
            successors.append(("descend", (monkey, box, 'ground', has)))
        if monkey == banana_loc and height == 'on_box':
            successors.append(("grasp", (monkey, box, height, True)))
    return successors

def bfs(start_state, banana_loc, locations):
    q = deque()
    q.append((start_state, []))
    visited = set()
    while q:
        state, path = q.popleft()
        if state[3]:
            return path, state
        if state in visited:
            continue
        visited.add(state)
        for action, new_state in gen_successors(state, banana_loc, locations):
            if new_state not in visited:
                q.append((new_state, path + [action]))
    return None, None

if __name__ == "__main__":
    locations = ['A', 'B', 'C']
    start_state = ('A', 'B', 'ground', False)
    banana_loc = 'C'
    plan, final_state = bfs(start_state, banana_loc, locations)
    if plan:
        for i, a in enumerate(plan, 1):
            print(f"{i}. {a}")
        print("Resulting state:", final_state)
    else:
        print("No plan found")